In [1]:
import numpy as np
from PIL import Image, ImageFilter
from skimage.io import imread, imsave
from skimage import img_as_float, draw
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import os
from scipy.ndimage import gaussian_filter
import sys
from queue import Queue
import cv2

In [2]:
def merge_blobs(equality_list):
    q_list = np.array(equality_list)
    equality_dict = dict()
    for q in q_list:
        if q[0] == q[1]:
            continue
        elif q[0] in equality_dict:
            equality_dict[q[0]].append(q[1])
            q_list[q_list == q[1]] = q[0]
        elif q[1] in equality_dict:
            equality_dict[q[1]].append(q[0])
            q_list[q_list == q[0]] = q[1]
        else:
            equality_dict[q[0]] = [q[1]]
            q_list[q_list == q[1]] = q[0]
    return equality_dict
    
def detect_objects(img, min_blob_size=100):
    h, w = img.shape
    img_visit = np.zeros((h+2,w+2))
    equality_list = []
    boxes = []
    blob_idx = 0
    msk = [[-1,-1], [-1,0], [-1, 1], [0,-1]]
    for i in range(1, h+1):
        for j in range(1, w+1):
            if img[i-1,j-1] != 0:
                for m, k in msk:
                    if img_visit[i,j] != img_visit[i+m, j+k] and img_visit[i+m, j+k]!= 0:
                        if img_visit[i,j] !=0:
                            equality_list.append(sorted([img_visit[i,j], img_visit[i+m, j+k]]))
                        img_visit[i,j] = img_visit[i+m, j+k]
                if img_visit[i,j] ==0:
                    blob_idx +=1
                    img_visit[i,j] = blob_idx
    equality_dict = merge_blobs(equality_list)
    for key in equality_dict:
        for idx in equality_dict[key]:
            img_visit[img_visit==idx] = key
    for key in equality_dict:
        blob_pixels = np.where(img_visit==key)
        if blob_pixels[0].shape[0] == 0:
            continue
        max_y, max_x = np.max(blob_pixels, axis=1)
        min_y, min_x = np.min(blob_pixels, axis=1)
        if max_x - min_x < min_blob_size or max_y - min_y < min_blob_size:
            continue
        boxes.append([min_y, min_x, max_y, max_x])
    return boxes,img_visit

def find_best_threshold(img):
    hist = np.histogram(img, bins=100, range=(0,1))[0]
    hist_min = (np.argmin(hist[30:70]) + 30)/100
    return hist_min

def filter_background(img, threshold):
    img_map = np.ones(img.shape)
    img_map[img<threshold] = 0
    return img_map

def detect_save(address, save_address,file_name):
    img_color = imread(address)
    img = rgb2gray(img_color)
    img = gaussian_filter(img, sigma=1)
    #ax = plt.hist(img.ravel(), bins = 256)
    t = find_best_threshold(img)
    img_map = filter_background(img, t)
    boxes, img_v = detect_objects(img_map)
    img2 = img_color.copy()
    img_folder = os.path.join(save_address, address[:-4])
    os.makedirs(img_folder, exist_ok=True)
    for i, box in enumerate(boxes):
        img2 = cv2.rectangle(img2, (box[1],box[0]), (box[3],box[2]), (0,0,0), 5)
        imsave(os.path.join(img_folder,file_name)+'_'+str(i)+'.jpg', img_color[box[0]:box[2], box[1]:box[3]])
    imsave(os.path.join(img_folder,file_name)+'.jpg', img_color)
    imsave(os.path.join(img_folder,file_name)+'_border_box.jpg', img2)
    
    
def batch_detect_object(address, save_address):
    for file_name in os.listdir(address):
        file_address = os.path.join(address, file_name)
        if os.path.isdir(file_address):
            os.makedirs(file_address, exist_ok=True)
            batch_detect_object(file_address, save_address)
        else:
            detect_save(file_address, save_address, file_name[:-4])

    
address = './foto'
save_address = './splited'
batch_detect_object(address, save_address)
